In [1]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [2]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-06-16 14:28:02--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.81.208, 172.217.8.16, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.81.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  77.5MB/s    in 0.8s    

2021-06-16 14:28:02 (77.5 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Pre procesado del data set

### Importar las dependencias del proyecto

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.5.0'

### Descomprimir el dataset de Perros vs Gatos

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [6]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [8]:
zip_object.extractall("./")

In [9]:
zip_object.close()

### Configurar las rutas al dataset

In [10]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [11]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [12]:
IMG_SHAPE = (128, 128, 3)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Congelar el modelo base

In [15]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [16]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [17]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [18]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [19]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [20]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

### Compilar el modelo

In [23]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
MobileNet soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [24]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [25]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [26]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [27]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
16/16 [==============================] - 41s 564ms/step - loss: 0.7112 - accuracy: 0.5750 - val_loss: 0.6561 - val_accuracy: 0.6150
Epoch 2/8
16/16 [==============================] - 7s 475ms/step - loss: 0.6066 - accuracy: 0.6710 - val_loss: 0.5749 - val_accuracy: 0.6930
Epoch 3/8
16/16 [==============================] - 8s 476ms/step - loss: 0.5307 - accuracy: 0.7420 - val_loss: 0.5089 - val_accuracy: 0.7450
Epoch 4/8
16/16 [==============================] - 7s 468ms/step - loss: 0.4687 - accuracy: 0.8005 - val_loss: 0.4538 - val_accuracy: 0.8040
Epoch 5/8
16/16 [==============================] - 7s 475ms/step - loss: 0.4171 - accuracy: 0.8350 - val_loss: 0.4070 - val_accuracy: 0.8370
Epoch 6/8
16/16 [==============================] - 7s 476ms/step - loss: 0.3721 - accuracy: 0.8650 - val_loss: 0.3669 - val_accuracy: 0.8670
Epoch 7/8
16/16 [==============================] - 7s 472ms/step - loss: 0.3354 - accuracy: 0.8825 - val_loss: 0.3335 - val_accuracy: 0.8880
Epoch 8/8
16

### Evaluar el modelo de aprendizaje por transferencia

In [28]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [29]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8999999761581421


## Puesta a punto de parámetros


Puntos importantes:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [30]:
base_model.trainable = True

In [31]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [32]:
fine_tune_at = 100

In [33]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [34]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [35]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 14s 560ms/step - loss: 0.1690 - accuracy: 0.9370 - val_loss: 0.1018 - val_accuracy: 0.9630
Epoch 2/5
16/16 [==============================] - 8s 475ms/step - loss: 0.0238 - accuracy: 0.9990 - val_loss: 0.1206 - val_accuracy: 0.9610
Epoch 3/5
16/16 [==============================] - 8s 475ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.1215 - val_accuracy: 0.9620
Epoch 4/5
16/16 [==============================] - 8s 479ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.1127 - val_accuracy: 0.9670
Epoch 5/5
16/16 [==============================] - 8s 477ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1341 - val_accuracy: 0.9620


### Evaluar el modelo re calibrado

In [36]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [37]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9620000123977661
